# K-nearest neighbours using different similarities and ensemble all K-nearest models

In [216]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
print("imported libraries . . . ")

imported libraries . . . 


In [217]:
#Read the data
train = pd.read_csv("/home/smita/MP/dataSet2014.csv")

#### What are the top booked hotel cluster in the dataset 

In [218]:
targets = pd.DataFrame(train.hotel_cluster.value_counts())
targets['Percentage'] = 100*targets['hotel_cluster']/train.shape[0]
targets.head()

,hotel_cluster,Percentage
91,727343,2.746410
41,546278,2.062718
48,518345,1.957244
64,488729,1.845416
65,475511,1.795505


In [219]:
train.shape

(26483412, 23)

# DownSampling

### We are seletcing data randomly for n unique users

In [220]:
### Randomly select n user_ids
import random
unique_users = train.user_id.unique()
print("We have data for", len(unique_users), "users")
usr_cnt = round(len(unique_users) * 0.75) # select 75% of the unique users
print("Randomly selecting data for", usr_cnt, "users")
sel_user_ids = [unique_users[i] for i in sorted(random.sample(range(len(unique_users)), usr_cnt)) ]
rf_dat = train[train.user_id.isin(sel_user_ids)]
print("new sample contains", rf_dat.shape[0], "rows and", rf_dat.shape[1], "columns")

We have data for 1092445 users
Randomly selecting data for 819334 users
new sample contains 19857217 rows and 23 columns


### We will split the data into train and test set

In [221]:
from sklearn import cross_validation
train_sample, test_sample = cross_validation.train_test_split(rf_dat, train_size = 0.75 )
print("Our train and test contains", train_sample.shape[0], ",", test_sample.shape[0], "rows respectivelly")
test_sample = test_sample[test_sample['is_booking']==1] ## using only booking data for test
print("Size of test sample after removal of clicks: ", test_sample.shape)

Our train and test contains 14892912 , 4964305 rows respectivelly
Size of test sample after removal of clicks:  (371058, 23)


In [146]:
train_sample.columns.values, test_sample.columns.values

(array(['Unnamed: 0', 'srch_destination_id', 'user_id', 'is_mobile',
        'is_package', 'is_booking', 'hotel_cluster', 'is_alone',
        'hotel_market', 'year', 'month', 'day', 'hour', 'part_of_day',
        'type_of_day', 'season', 'duration_of_stay', 'hotel_market.1',
        'hotel_continent', 'X', 'dest_feature_pc1', 'dest_feature_pc2',
        'dest_feature_pc3'], dtype=object),
 array(['Unnamed: 0', 'srch_destination_id', 'user_id', 'is_mobile',
        'is_package', 'is_booking', 'hotel_cluster', 'is_alone',
        'hotel_market', 'year', 'month', 'day', 'hour', 'part_of_day',
        'type_of_day', 'season', 'duration_of_stay', 'hotel_market.1',
        'hotel_continent', 'X', 'dest_feature_pc1', 'dest_feature_pc2',
        'dest_feature_pc3'], dtype=object))

In [222]:
### deleting unwanted index columns from test and train
del train_sample['Unnamed: 0']
del test_sample['Unnamed: 0']

In [223]:
train_sample.head(1) 

,srch_destination_id,user_id,is_mobile,is_package,is_booking,hotel_cluster,is_alone,hotel_market,year,month,...,part_of_day,type_of_day,season,duration_of_stay,hotel_market.1,hotel_continent,X,dest_feature_pc1,dest_feature_pc2,dest_feature_pc3
10218408,8746,566955,0,0,0,59,0.0,29,2014,10,...,evening,weekday,fall,2.0,29,6,8526.0,7.301992,-2.167246,1.25915


In [224]:
test_sample.head(1)

,srch_destination_id,user_id,is_mobile,is_package,is_booking,hotel_cluster,is_alone,hotel_market,year,month,...,part_of_day,type_of_day,season,duration_of_stay,hotel_market.1,hotel_continent,X,dest_feature_pc1,dest_feature_pc2,dest_feature_pc3
15131096,11922,815699,0,1,1,37,0.0,376,2014,9,...,morning,weekday,fall,1.0,376,2,11525.0,12.146671,-0.228022,3.466711


### Selecting only required columns to free some memory

In [225]:
predictor_set = [c for c in train.columns if c in ['srch_destination_id', 'hotel_continent', 'hotel_market', 'hotel_cluster']]

In [226]:
train_noDup  = train_sample[predictor_set] 
print("size of train set before removal of suplicates:", train_noDup.shape)
# We are dropping duplicate rows from training set 
train_noDup = train_noDup.drop_duplicates(subset=predictor_set)
print("size of train set after removal of suplicates:", train_noDup.shape)

size of train set before removal of suplicates: (14892912, 4)
size of train set after removal of suplicates: (300985, 4)


#### Separate predictors and target from test and train set

In [227]:
predictor_set1 = [c for c in train.columns if c in ['srch_destination_id', 'hotel_continent', 'hotel_market']]
train_dat = train_noDup[predictor_set1]
train_label = train_noDup['hotel_cluster']
test_dat = test_sample[predictor_set1]
test_label = test_sample['hotel_cluster']

### before applying KNN lets delete all

#### We will Jaccard similarty Score to find 5 nearest neighbours to predict what hotel clusters booked by the users in test set

In [228]:
from itertools import islice
def sliceDict(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [229]:
from sklearn.metrics import jaccard_similarity_score
import operator
import gc # to free memory

In [ ]:
distances = {} # {hotel_cluster: score}
final_predction = []
nrowtrain,ncoltrain = train_dat.shape
nrowtest,ncoltest = test_dat.shape
for testrow in range(nrowtest):
    
        for r in range(nrowtrain):
            score = jaccard_similarity_score(train_dat.iloc[r,], test_dat.iloc[testrow,])
            hotel_cluster = train_label.iloc[r] # get hotel_cluster of neighbour
            distances.update({hotel_cluster: score})
            
        sorted_distances = sorted(distances.items(), key=operator.itemgetter(1))
        top5 = sliceDict(5, sorted_distances)
        p =[]
        for i in range(len(top5)):
                            d = top5[i][0]
                            p.append(d)
        #print(p)            
        final_predction.append(p)   
       

# Accuracy Using Map@5 metric

In [213]:
import ml_metrics as metrics
target = [[l] for l in test_sample["hotel_cluster"]]
score = metrics.mapk(target, final_predction, k=5) 
score*100

8.75

In [ ]:
from sklearn import neighbors, datasets
